In [ ]:
import time

import pandas as pd

from collections import defaultdict

import spotipy
from spotipy.oauth2 import SpotifyClientCredentials

from dotenv import load_dotenv

import os

# ---<---<--<--@    hasta aquí nuestras librerías   @-->-->--->---

In [4]:

load_dotenv()
client_id = os.getenv("CLIENT_ID")
client_secret = os.getenv("CLIENT_SECRET")

# Autenticación con Spotify
auth_manager = SpotifyClientCredentials(client_id=client_id, client_secret=client_secret)
sp = spotipy.Spotify(auth_manager=auth_manager)

def extraer_datos_por_genero_y_rango(genero, rango, resultado_anual=200):
    albumes_por_anyo = defaultdict(list)
    canciones_por_anyo = defaultdict(list)
    for o in range(0, 200, 50):
        # Pausa para evitar rate limits
        time.sleep(0.5)
        artistas = sp.search(q=f'genre:{genero}', type='artist', limit=50, offset=o)
        for artista in artistas['artists']['items']:
            artista_id = artista['id']
            artista_nombre = artista['name']

            # Extraemos álbumes
            albumes_data = sp.artist_albums(artista_id, album_type=['album'], limit=50)
            for album in albumes_data['items']:
                try:
                    album_lanzamiento = int(album['release_date'][:4])

                    if album_lanzamiento in rango and len(albumes_por_anyo[album_lanzamiento]) < resultado_anual:
                        albumes_por_anyo[album_lanzamiento].append({
                            'artista': artista_nombre,
                            'artista_id': artista_id,  
                            'genero': genero,
                            'tipo': album['album_type'],
                            'nombre': album['name'],
                            'año': album_lanzamiento,
                            'popularidad_artista': artista['popularity']
                        })
                except (KeyError, ValueError, TypeError):
                    continue

            # Extraemos canciones
            try:
                top_tracks = sp.artist_top_tracks(artista_id)
                for track in top_tracks['tracks']:
                    try:
                        track_lanzamiento = int(track['album']['release_date'][:4])
                        if track_lanzamiento in rango and len(canciones_por_anyo[track_lanzamiento]) < resultado_anual:
                            canciones_por_anyo[track_lanzamiento].append({
                                'artista': artista_nombre,
                                'artista_id': artista_id,  # ID del artista
                                'genero': genero,
                                'tipo': 'cancion',
                                'nombre': track['name'],
                                'año': track_lanzamiento,
                                'id': track['id'],  # ID de la canción
                                'popularidad_cancion': track['popularity']
                            })
                    except (KeyError, ValueError, TypeError):
                        continue
            except Exception as e:
                print(f"Error al obtener top tracks: {e}")
                continue

    return {
        'albumes': [{'tipo': 'album', **data} for album_lanzamiento in rango for data in albumes_por_anyo[album_lanzamiento][:resultado_anual]],
        'canciones': [{'tipo': 'cancion', **data} for anyo in rango for data in canciones_por_anyo[anyo][:resultado_anual]]
    }


In [5]:
# Uso: 
# LLAMADA EXTRACCIÓN -> género COUNTRY 

tastyinfo_2018_2022 = extraer_datos_por_genero_y_rango('country', range(2018, 2023))
print(f'\nLlamada a Spotify (( ☎ )) Spotify ha respondido\n    . . . . . . . . . . . . . . . . . . .\n\nHemos obtenido {len(tastyinfo_2018_2022)} listas ☑\n')

albumes_2018_2022 = tastyinfo_2018_2022['albumes']  # separar una lista de álbumes
print(f'Se ha creado una lista con la info de los {len(albumes_2018_2022)} álbumes ☑')
canciones_2018_2022 = tastyinfo_2018_2022['canciones']  # separar una lista de canciones
print(f'Se ha creado una lista con la info de las {len(canciones_2018_2022)} canciones ☑\n')

# Conversión a Data Frame:

df_2018_2022_album = pd.DataFrame(albumes_2018_2022)
print(f'DataFrame álbumes del género {albumes_2018_2022[0]["genero"]} ☑\n')
df_2018_2022_tracks = pd.DataFrame(canciones_2018_2022)
print(f'DataFrame top tracks del género {albumes_2018_2022[1]["genero"]} ☑\n')

# Paso de exportación (de DataFrame a .csv):
# 
# Cada una tenemos que modificar el nombre que queremos para el archivo según su género 

#                         ---<-----<-----@ archivo para albumes a .CSV @----->---->---

df_2018_2022_album.to_csv('../csv/generos/country_albumes_2018_2022.csv', index=False, encoding='utf-8')
print("""
ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")

#                       --<-----<-----@ archivo para canciones a .CSV @----->---->---

df_2018_2022_tracks.to_csv('../csv/generos/country_top_tracks_2018_2022.csv', index=False, encoding='utf-8')
print("""
TOP TRACKS              2018                                              
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022              
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")




Llamada a Spotify (( ☎ )) Spotify ha respondido
    . . . . . . . . . . . . . . . . . . .

Hemos obtenido 2 listas ☑

Se ha creado una lista con la info de los 495 álbumes ☑
Se ha creado una lista con la info de las 381 canciones ☑

DataFrame álbumes del género country ☑

DataFrame top tracks del género country ☑


ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--             

In [ ]:
# USO 
# LLAMADA EXTRACCIÓN -> género METAL

tastyinfo_2018_2022 = extraer_datos_por_genero_y_rango('metal', range(2018, 2023))
print(f'\nLlamada a Spotify (( ☎ )) Spotify ha respondido\n    . . . . . . . . . . . . . . . . . . .\n\nHemos obtenido {len(tastyinfo_2018_2022)} listas ☑\n')

albumes_2018_2022 = tastyinfo_2018_2022['albumes']  # separar una lista de álbumes
print(f'Se ha creado una lista con la info de los {len(albumes_2018_2022)} álbumes ☑')
canciones_2018_2022 = tastyinfo_2018_2022['canciones']  # separar una lista de canciones
print(f'Se ha creado una lista con la info de las {len(canciones_2018_2022)} canciones ☑\n')

# Conversión a Data Frame:

df_2018_2022_album = pd.DataFrame(albumes_2018_2022)
print(f'DataFrame álbumes del género {albumes_2018_2022[0]["genero"]} ☑\n')
df_2018_2022_tracks = pd.DataFrame(canciones_2018_2022)
print(f'DataFrame top tracks del género {albumes_2018_2022[1]["genero"]} ☑\n')

#                         ---<-----<-----@ archivo para albumes a .CSV @----->---->---

df_2018_2022_album.to_csv('../../generos/metal_albumes_2018_2022.csv', index=False, encoding='utf-8')
print("""
ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")

#                       --<-----<-----@ archivo para canciones a .CSV @----->---->---

df_2018_2022_tracks.to_csv('../../generos/metal_top_tracks_2018_2022.csv', index=False, encoding='utf-8')
print("""
TOP TRACKS              2018                                              
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022              
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")


Llamada a Spotify (( ☎ )) Spotify ha respondido
    . . . . . . . . . . . . . . . . . . .

Hemos obtenido 2 listas ☑

Se ha creado una lista con la info de los 523 álbumes ☑
Se ha creado una lista con la info de las 367 canciones ☑

DataFrame álbumes del género metal ☑

DataFrame top tracks del género metal ☑


ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                 

In [ ]:
#   USO
#   LLAMADA EXTRACCIÓN -> género POP

tastyinfo_2018_2022 = extraer_datos_por_genero_y_rango('pop', range(2018, 2023))
print(f'\nLlamada a Spotify (( ☎ )) Spotify ha respondido\n    . . . . . . . . . . . . . . . . . . .\n\nHemos obtenido {len(tastyinfo_2018_2022)} listas ☑\n')

albumes_2018_2022 = tastyinfo_2018_2022['albumes']  # separar una lista de álbumes
print(f'Se ha creado una lista con la info de los {len(albumes_2018_2022)} álbumes ☑')
canciones_2018_2022 = tastyinfo_2018_2022['canciones']  # separar una lista de canciones
print(f'Se ha creado una lista con la info de las {len(canciones_2018_2022)} canciones ☑\n')

# Conversión a Data Frame:

df_2018_2022_album = pd.DataFrame(albumes_2018_2022)
print(f'DataFrame álbumes del género {albumes_2018_2022[0]["genero"]} ☑\n')
df_2018_2022_tracks = pd.DataFrame(canciones_2018_2022)
print(f'DataFrame top tracks del género {albumes_2018_2022[1]["genero"]} ☑\n')

# Paso de exportación (de DataFrame a .csv):
# 
# Cada una tenemos que modificar el nombre que queremos para el archivo según su género 

#                         ---<-----<-----@ archivo para albumes a .CSV @----->---->---

df_2018_2022_album.to_csv('../../generos/pop_albumes_2018_2022.csv', index=False, encoding='utf-8')
print("""
ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")

#                       --<-----<-----@ archivo para canciones a .CSV @----->---->---

df_2018_2022_tracks.to_csv('../../generos/pop_top_tracks_2018_2022.csv', index=False, encoding='utf-8')
print("""
TOP TRACKS              2018                                              
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022              
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")


Llamada a Spotify (( ☎ )) Spotify ha respondido
    . . . . . . . . . . . . . . . . . . .

Hemos obtenido 2 listas ☑

Se ha creado una lista con la info de los 490 álbumes ☑
Se ha creado una lista con la info de las 394 canciones ☑

DataFrame álbumes del género pop ☑

DataFrame top tracks del género pop ☑


ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                     

In [ ]:
#   USO 
#   LLAMADA EXTRACIÓN -> género ROCK

tastyinfo_2018_2022 = extraer_datos_por_genero_y_rango('rock', range(2018, 2023))
print(f'\nLlamada a Spotify (( ☎ )) Spotify ha respondido\n    . . . . . . . . . . . . . . . . . . .\n\nHemos obtenido {len(tastyinfo_2018_2022)} listas ☑\n')

albumes_2018_2022 = tastyinfo_2018_2022['albumes']  # separar una lista de álbumes
print(f'Se ha creado una lista con la info de los {len(albumes_2018_2022)} álbumes ☑')
canciones_2018_2022 = tastyinfo_2018_2022['canciones']  # separar una lista de canciones
print(f'Se ha creado una lista con la info de las {len(canciones_2018_2022)} canciones ☑\n')

# Conversión a Data Frame:

df_2018_2022_album = pd.DataFrame(albumes_2018_2022)
print(f'DataFrame álbumes del género {albumes_2018_2022[0]["genero"]} ☑\n')
df_2018_2022_tracks = pd.DataFrame(canciones_2018_2022)
print(f'DataFrame top tracks del género {albumes_2018_2022[1]["genero"]} ☑\n')


#                         ---<-----<-----@ archivo para albumes a .CSV @----->---->---

df_2018_2022_album.to_csv('../../generos/rock_albumes_2018_2022.csv', index=False, encoding='utf-8')
print("""
ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")

#                       --<-----<-----@ archivo para canciones a .CSV @----->---->---

df_2018_2022_tracks.to_csv('../../generos/rock_top_tracks_2018_2022.csv', index=False, encoding='utf-8')
print("""
TOP TRACKS              2018                                              
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022              
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")


Llamada a Spotify (( ☎ )) Spotify ha respondido
    . . . . . . . . . . . . . . . . . . .

Hemos obtenido 2 listas ☑

Se ha creado una lista con la info de los 538 álbumes ☑
Se ha creado una lista con la info de las 135 canciones ☑

DataFrame álbumes del género rock ☑

DataFrame top tracks del género rock ☑


ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                   

In [ ]:
#   USO
#   LLAMADA EXTRACCIÓN -> género r&b

tastyinfo_2018_2022 = extraer_datos_por_genero_y_rango('r&b', range(2018, 2023))
print(f'\nLlamada a Spotify (( ☎ )) Spotify ha respondido\n    . . . . . . . . . . . . . . . . . . .\n\nHemos obtenido {len(tastyinfo_2018_2022)} listas ☑\n')

albumes_2018_2022 = tastyinfo_2018_2022['albumes']  # separar una lista de álbumes
print(f'Se ha creado una lista con la info de los {len(albumes_2018_2022)} álbumes ☑')
canciones_2018_2022 = tastyinfo_2018_2022['canciones']  # separar una lista de canciones
print(f'Se ha creado una lista con la info de las {len(canciones_2018_2022)} canciones ☑\n')

# Conversión a Data Frame:

df_2018_2022_album = pd.DataFrame(albumes_2018_2022)
print(f'DataFrame álbumes del género {albumes_2018_2022[0]["genero"]} ☑\n')
df_2018_2022_tracks = pd.DataFrame(canciones_2018_2022)
print(f'DataFrame top tracks del género {albumes_2018_2022[1]["genero"]} ☑\n')

#                         ---<-----<-----@ archivo para albumes a .CSV @----->---->---

df_2018_2022_album.to_csv('../../generos/r&b_albumes_2018_2022.csv', index=False, encoding='utf-8')
print("""
ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")

#                       --<-----<-----@ archivo para canciones a .CSV @----->---->---

df_2018_2022_tracks.to_csv('../../generos/r&b_top_tracks_2018_2022.csv', index=False, encoding='utf-8')
print("""
TOP TRACKS              2018                                              
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022              
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                      --====--         ---======--- \n""")


Llamada a Spotify (( ☎ )) Spotify ha respondido
    . . . . . . . . . . . . . . . . . . .

Hemos obtenido 2 listas ☑

Se ha creado una lista con la info de los 382 álbumes ☑
Se ha creado una lista con la info de las 433 canciones ☑

DataFrame álbumes del género r&b ☑

DataFrame top tracks del género r&b ☑


ALBUMES                 2018                                           
  `b                ..rmMMbmy..                                 
   `b            .dMP"'     `"VMb.                              
    `b         .p'              `Mb              2022               
     `b      ,p'                 `Mb           ,mdMMbm.         
      q.   ,dP                     `b       ,MP"'   `"MMb.      
      `b  .P                        `b    ,P'           `b.     
       `L M                          `L ,P          *** .CSV ***
        MM                            MM'                 ☑
        `P                            `P                        
     --====--                     